Notebook dédié à l'industrialisation de ce que j'ai construit:
- dashboard avec des viz et des infos générales et par club
- plateforme qui permet de générer une prédiction d'un match entre 2 équipes

Structuration du dashboard:
- une page générale:
   - les derniers matchs recensés dans la base (train + test bien sûr, et globalement raise une erreur si on n'est pas à jour)
   - le classement actuel avec nombre de matches joués, nombre de points, nombre de buts marqués, nombre de buts encaissés, différence de buts
   - classement sur les 5 derniers matvhs
   - une viz qui montre les équipes en forme du moment
   - une viz qui montre les équipes en méforme du moment
 
- une page par équipe (on filtre l'équipe que l'on veut):
   - les derniers matches de l'équipe en question
   - une viz qui montre la forme du moment
   - le classement actuel de l'équipe
   - l'évolution du classement de la saison en cours
   - plus large victoire de la saison en cours
   - plus large défaite de la saison en cours

- une page prédiction : je choisis mes deux équipes, mes deux modèles, et ça me dit le score qu'il voit. Mettre une espèce de heatmap ou distribution qui répartie les probas d'avoir tel ou tel résultat, pouis tel ou tel score

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from joblib import load

from ipywidgets import HBox, VBox, HTML, Dropdown, Button, Output, Tab
from sklearn.preprocessing import StandardScaler

root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

from src.config import load_config
from src.modeling import load_model

# config.yaml importation
config_file = 'config.yaml'
config_path = os.path.join(root_path, config_file)
config = load_config(config_path)

In [2]:
home_poisson = load_model(os.path.join('..', config['secondary_models_dir'], 'home_poisson.joblib'))
away_poisson = load_model(os.path.join('..', config['secondary_models_dir'], 'away_poisson.joblib'))
primary_rf = load_model(os.path.join('..', config['primary_models_dir'], 'rf.joblib'))

Model loaded from ..\models/secondary\home_poisson.joblib
Model loaded from ..\models/secondary\away_poisson.joblib
Model loaded from ..\models/primary\rf.joblib


In [3]:
pre = home_poisson.named_steps["pre"]

num_cols = pre.transformers_[0][2]
cat_cols = pre.transformers_[1][2]

print("Numériques :", num_cols)
print("Catégorielles :", cat_cols)

# Charger ton dataset de features finales
df_features = pd.read_csv("../data/preprocessed/preprocessed_df_train.csv")

teams = sorted(df_features[config['home_column']].unique())

Numériques : ['current_season_home_team_ranking_at_home', 'current_season_nb_points_home_team_at_home', 'current_season_nb_goals_scored_home_team_at_home', 'current_season_nb_goals_conceded_home_team_at_home', 'current_season_general_ranking_home_team', 'current_season_nb_goals_conceded_home_team', 'current_season_goal_difference_home_team', 'current_season_attack_ranking_home_team', 'current_season_defense_ranking_home_team', 'abs_recent_nb_points_by_match_home_team', 'abs_recent_nb_goals_scored_by_match_home_team', 'abs_recent_nb_goals_conceded_by_match_home_team', 'abs_recent_goal_difference_home_team', 'abs_recent_ranking_home_team', 'abs_hist_nb_points_by_season_home_team', 'abs_hist_nb_goals_conceded_by_season_home_team', 'abs_hist_ranking_by_season_home_team', 'rel_recent_nb_points_by_match_home_team', 'rel_recent_nb_goals_scored_by_match_home_team', 'rel_recent_nb_goals_conceded_by_match_home_team', 'rel_recent_goal_difference_home_team', 'rel_recent_percentage_victory_home_tea

In [4]:
teams

['Ajaccio',
 'Amiens',
 'Angers',
 'Arles',
 'Auxerre',
 'Bastia',
 'Bordeaux',
 'Brest',
 'Caen',
 'Clermont',
 'Dijon',
 'ETG',
 'Gazélec Ajaccio',
 'Guingamp',
 'Le Havre',
 'Lens',
 'Lille',
 'Lorient',
 'Lyon',
 'Marseille',
 'Metz',
 'Monaco',
 'Montpellier',
 'Nancy',
 'Nantes',
 'Nice',
 'Nimes',
 'PSG',
 'Reims',
 'Rennes',
 'Saint-Etienne',
 'Sochaux',
 'Strasbourg',
 'Toulouse',
 'Troyes',
 'Valenciennes']

In [5]:
def build_input_row(home_team, away_team):
    """
    Construit la ligne d'entrée modèle (home/away features)
    """
    row = df_features[df_features[config['home_column']] == home_team].iloc[-1:].copy()

    row[config['home_column']] = home_team
    row[config['away_column']] = away_team

    return row


def predict_1x2(row, model):
    proba = model.predict_proba(row)[0]
    classes = model.classes_.tolist()
    return {
        "Home" : proba[classes.index("H")],
        "Draw" : proba[classes.index("D")],
        "Away" : proba[classes.index("A")],
    }


def predict_score(row, model_home, model_away):
    home = model_home.predict(row)[0]
    away = model_away.predict(row)[0]
    return home, away


from scipy.stats import poisson

def poisson_matrix(lh, la, max_goals=5):
    mat = np.zeros((max_goals+1, max_goals+1))
    for i in range(max_goals+1):
        for j in range(max_goals+1):
            mat[i,j] = poisson.pmf(i, lh) * poisson.pmf(j, la)
    return mat


In [6]:
dashboard_output = Output()

def dashboard_page():
    with dashboard_output:
        dashboard_output.clear_output()
        fig, ax = plt.subplots(1,2, figsize=(12,4))

        df_goals = df_features.groupby(config['home_column'])[[config['nb_goals_home_column'], config['nb_goals_away_column']]].mean()

        df_goals[config['nb_goals_home_column']].sort_values().plot(kind="barh", ax=ax[0], title="Buts marqués à domicile (moyenne)")
        df_goals[config['nb_goals_away_column']].sort_values().plot(kind="barh", ax=ax[1], title="Buts encaissés à domicile (moyenne)")

        plt.show()

dashboard_page()


In [7]:
team_selector = Dropdown(
    options=teams,
    description="Équipe:",
    value=teams[0]
)

team_output = Output()

def update_team_dashboard(change=None):
    team = team_selector.value
    df = df_features[df_features[config['home_column']] == team].tail(15)

    with team_output:
        team_output.clear_output()

        fig, ax = plt.subplots(1,1, figsize=(10,4))
        ax.plot(df[config['nb_goals_home_column']].values, marker='o')
        ax.set_title(f"Buts marqués par {team} (15 derniers matchs)")
        plt.show()

team_selector.observe(update_team_dashboard, names="value")
update_team_dashboard()

In [8]:
home_team_dd = Dropdown(options=teams, description="Domicile:")
away_team_dd = Dropdown(options=teams, description="Extérieur:")
model_1x2_dd = Dropdown(options=["Logistic", "RandomForest", "XGBoost"], description="Modèle 1X2:")
model_score_dd = Dropdown(options=["Poisson", "RandomForest", "XGBoost"], description="Modèle score:")

predict_btn = Button(description="Prédire", button_style="success")
predict_output = Output()


def on_predict_clicked(b):
    ht = home_team_dd.value
    at = away_team_dd.value
    m1 = model_1x2_dd.value
    m2 = model_score_dd.value

    row = build_input_row(ht, at)

    # choix modèle 1X2
    if m1 == "Logistic":
        model_1x2 = primary_log
    elif m1 == "RandomForest":
        model_1x2 = primary_rf
    else:
        model_1x2 = primary_xgb

    proba = predict_1x2(row, model_1x2)

    # choix modèle score
    if m2 == "Poisson":
        h, a = predict_score(row, home_poisson, away_poisson)
    elif m2 == "RandomForest":
        h, a = predict_score(row, home_rf, away_rf)
    else:
        h, a = predict_score(row, home_xgb, away_xgb)

    with predict_output:
        predict_output.clear_output()
        print(f"Prédiction {ht} – {at}")
        print("Probabilités:")
        print(f"  Victoire domicile : {proba['Home']:.2%}")
        print(f"  Nul               : {proba['Draw']:.2%}")
        print(f"  Victoire extérieur: {proba['Away']:.2%}")
        print("\nScore attendu:")
        print(f"  {ht} {h:.2f} – {a:.2f} {at}")

predict_btn.on_click(on_predict_clicked)


In [9]:
tab = Tab()

tab.children = [
    dashboard_output,
    VBox([team_selector, team_output]),
    VBox([home_team_dd, away_team_dd, model_1x2_dd, model_score_dd, predict_btn, predict_output])
]

tab.set_title(0, "Dashboard général")
tab.set_title(1, "Par équipe")
tab.set_title(2, "Prédiction de match")

tab


In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widgets
home_select = widgets.Dropdown(options=['Team A', 'Team B', 'Team C'], description="Home:")
away_select = widgets.Dropdown(options=['Team A', 'Team B', 'Team C'], description="Away:")
btn_predict = widgets.Button(description="Prédire", button_style='success')
out = widgets.Output()

# Callback
def on_predict_clicked(b):
    with out:
        out.clear_output()
        print("Bouton cliqué !")
        print(f"Home = {home_select.value}, Away = {away_select.value}")
        print("→ Ici tu mets ta prédiction")

# Binding
btn_predict.on_click(on_predict_clicked)

# Layout
ui = widgets.VBox([home_select, away_select, btn_predict, out])

display(ui)
